### Why Seq. Modelling

Examples:
- Speach recognition
- Music generation
- Sentiment classification
- DNA sequence analysis
- Machine translation
- Video activity recognition
- Name entity recognition 

Output or input may be sequence only

### Notation

Consider a sentnece (seq. of words)  
$x$ is a sentence. 
Find where are people's names in the problem: 
`Named-entity recognition` used by search engines (indexing the words for fast search).  

Do one-hot encoding for names and not names (labels are 1 or 0, $y$-vector)

Indexes are $x^{<i>}$ or $x^{<t>}$ with length $T_x$ for temporal index  
and outputs are $y^{<t>}$ with length $T_y$.  

For various sets the length can be different. 
In particular $T_x^{(i)}$ and $T_y^{(i)}$

Representing words is done making a dictionary or vocabluary. 

Dictionaries are set of indexed words with large number of them. Then, each word is a vector the length of the whole vocabluary with one-hot 1 for this word and 0 for the rest. These are `one-hot` vectors. 

The goal is to learn mapping the mapping to the target output $Y$ using this one-hot vector via `supervised learning`. 

If the word is not in the vocabluary, use a  `UNK` (not in the vocabluary workd)


